In [2]:
from langchain_core.output_parsers  import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser
chain.invoke({"topic": "AI"})

"Why did the artificial intelligence break up with its robot girlfriend? It couldn't handle her constant buffering."

In [5]:
print(prompt.invoke({"topic":"AI"}))

messages=[HumanMessage(content='tell me a short joke about AI')]


In [7]:
from langchain_core.messages.human import HumanMessage
messages =[HumanMessage(content='tell me a short joke about AI')]
model.invoke(messages)

AIMessage(content='Why did the AI break up with its computer program girlfriend? \nBecause she had too many bugs in her code!', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 14, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-534361ce-a166-469d-8749-574591501f1b-0', usage_metadata={'input_tokens': 14, 'output_tokens': 23, 'total_tokens': 37})

In [4]:
from abc import ABC, abstractmethod

class CRunnable(ABC):
    def __init__(self):
        self.next = None

    @abstractmethod
    def process(self,data):
        """
        This method should be overridden by subclasses to implement the specific processing logic.
        """
    def invoke(self,data):
        processed_data = self.process(data)
        if self.next is not None:
            self.next.invoke(processed_data)
        
        return processed_data
    
    def __or__(self, other) :
        return CRunnableSequence(self, other)   
    
class CRunnableSequence(CRunnable):
    def __init__(self, first,second):
        super().__init__()
        self.first = first
        self.second = second
        
    def process(self, data):
        return data
    
    def invoke(self, data):
        first_result = self.first.invoke(data)
        return self.second.invoke(first_result)

In [5]:
class AddTen(CRunnable):
    def process(self,data):
        print("AddTen", data)
        return data + 10
    
class MultiplyByTwo(CRunnable):
    def process(self, data):
        print("MultiplyByTwo", data)
        return data * 2
    
class ConvertToString(CRunnable):
    def process(self, data):
        print("ConvertToString", data)
        return f'Result:{data}'

In [6]:
a=AddTen()
b=MultiplyByTwo()
c=ConvertToString()

chain = a|b|c

In [7]:
result = chain.invoke(10)
print(result)

AddTen 10
MultiplyByTwo 20
ConvertToString 40
Result:40


In [9]:
# Now change the sequence and check the chain
chain = b|a|c
result = chain.invoke(2)
print(result)

MultiplyByTwo 2
AddTen 4
ConvertToString 14
Result:14


## Runnables from Langchain

In [10]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

In [11]:
#RunnablePassthrough just pass the input and does nothing
#it is used with other Runnable to create a pipeline

chain = RunnablePassthrough() | RunnablePassthrough() | RunnablePassthrough()
chain.invoke("Hello")

'Hello'

In [12]:
# custom function to convert string to upper
def string_to_upper(input_string):
    """
    Converts a given string to uppercase.

    Args:
        input_string (str): The string to be converted to uppercase.

    Returns:
        str: The uppercase version of the input string.
    """
    return input_string.upper()

In [13]:
chain = RunnablePassthrough() | RunnableLambda(string_to_upper) | RunnablePassthrough()
chain.invoke("hello world")

'HELLO WORLD'

## RunnableParallel

In [14]:
chain = RunnableParallel({"x":RunnablePassthrough(),"y":RunnablePassthrough()})

In [16]:
chain.invoke("hello")

{'x': 'hello', 'y': 'hello'}

In [17]:
chain.invoke({"input1":"Hello","input2":"World"})

{'x': {'input1': 'Hello', 'input2': 'World'},
 'y': {'input1': 'Hello', 'input2': 'World'}}

In [18]:

chain = RunnableParallel({"x":RunnablePassthrough(),"y":lambda z:z["input2"]})
chain.invoke({"input1":1, "input2":2})

{'x': {'input1': 1, 'input2': 2}, 'y': 2}

## Nested Chains

In [20]:
def find_keys_to_uppercase(input:dict):
    output = input.get("input","not found").upper()
    return output

In [21]:
chain = RunnableParallel({"x":RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase),"y":lambda z:z["input2"]})
chain.invoke({"input1":"hello", "input2":"world"})

{'x': 'NOT FOUND', 'y': 'world'}

In [22]:
chain.invoke({"input":"hello", "input2":"world"})

{'x': 'HELLO', 'y': 'world'}

In [33]:
# Using FAISS with Lambda

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

In [34]:
vectorStoreFaiss  = FAISS.from_texts(['Gooseberry is better than Apple','Dog loves bones'],
                                     embedding=OpenAIEmbeddings()
                                     )

In [36]:
retriever = vectorStoreFaiss.as_retriever()
template = """Answer based on context:\n\n{context}\n\nQuestion: {question}\n\nAnswer:"""
prompt = ChatPromptTemplate.from_template(template=template)


In [37]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [38]:
rag_chain = (
    {"context":retriever | format_docs, "question":RunnablePassthrough()}
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)

In [39]:
rag_chain.invoke("Which is better than Apple?")

'Gooseberry'